# Generating Assets with Grok 🤖
___

In [226]:
import os
from dotenv import load_dotenv
load_dotenv()
from grok import Grok
import pickle

In [227]:
os.chdir(os.getenv("APPDIR"))

In [228]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
print(grok_api_key, grok_url)
grok = Grok(grok_api_key, grok_url, grok_model)

xai-fq2OPXVKJrQCMyitw8z4fUiuVE6OO0KBjy1AzT5L56DiZsG88SqNJgvISwXC7psVI6wmT5vkAkFWadXF https://api.x.ai/v1/chat/completions


In [229]:
def generate_script_grok(client: Grok, trend_data: dict, news_report: str) -> str:

    """Generate a video script using Grok based on trend data"""
    
    prompt = f"""Create a short, video script about this trending topic:
    Topic: {trend_data['keyword']}
    Related Keywords: {', '.join(trend_data['keywords'])}
    News Report: {news_report}
    
    The script must be:
    1. Informative and accurate
    2. Focused on the most important and sensational aspects of the story
    3. Conversational and concise.
    4. Written with charged language and emotional appeals.
    5. Quick to the point, no fluff.
    6. Between 60 and 80 words in length.

    The script must NOT:
    1. Contain an outro of any kind, or mention subscribing to the channel
    2. Use acronyms or abbreviations without explanation
    3. Contain any headers, formatting, directions for the video, or other instructions.

    End the script with a uniqueand engaging question.
    """
    return client.generate_text(prompt)

In [230]:
def generate_sora_prompts_grok(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    3. Use the newline character, never the word "newline"
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [231]:
def generate_sora_prompts_grokV2(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Scene descriptions should tell a story. Ensure that it can be told in ten seconds or less.
    6. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [232]:
def generate_grok_image_prompts(client: Grok, script: str, num_prompts: int = 10) -> list[str]:
    """Generate Grok image prompts based on the script"""

    prompt = f"""Generate a series of (~{int}) image descriptions that capture the most sensational themes of the following script.

    For each image:
    1. Be assertive and highly creative in your description. 
    2. Be clear and concise. 
    3. Use familiar elements of popular culture where appropriate e.g. brands, celebrities, landmarks, products, etc.

    Each image prompt should have the following format:
    Generate an image of [description]

    Separate each image prompt with a newline.
    
    Script: {script}
    """
    return client.generate_text(prompt)

In [233]:
def generate_grok_narration(client: Grok, script: str) -> str:
    """Generate a narration using Grok based on the script"""

    prompt = f"""Extract only the words to be spoken from this video script:
    Script: {script}.

    The result should NOT include any mention of a narrator or scene descriptions.
    """
    return grok.generate_text(prompt)

In [234]:
def generate_hashtags(client: Grok, script: str) -> list[str]:
    """Generate hashtags using Grok based on the script"""

    prompt = f"""Generate the best series of hashtags to make this video go viral on instagram reels, tik tok, and youtube shorts.
    Select the best 10 hashtags and brief reasoning for each.

    The result should be in the following format:
    for each hashtag:
        [Hashtag] [Reasoning]

    Include a one-line space separated list of all hashtags at the end.

    Script: {script}"""
    return client.generate_text(prompt)


In [235]:
def generate_grok_opinion_analysis(client: Grok, news_report: str) -> str:
    """Generate an opinion analysis using Grok based on the news report"""

    prompt = f"""Generate an opinion analysis using Grok based on the news report:
    News Report: {news_report}

    The opinion analysis should:
    1. Be highly engaging and bold.
    2. Provide evidence based reasoning for claims made. 
    3. Be less than 500 words in length.
    4. Be written in a professional tone.

    The opinion analysis should NOT:
    1. Cite sources.

    Format:
    [Title Highlighting the Central Claim] <= 10 words
    [Opinion Analysis] <= 500 words
    """

    return client.generate_text(prompt)

In [236]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
grok = Grok(grok_api_key, grok_url, grok_model)

In [237]:
# List directories in assets and get most recent
asset_dirs = [d for d in os.listdir('assets') if os.path.isdir(os.path.join('assets', d))]
RUN_DIR = os.path.join('assets', max(asset_dirs))
MAIN = [os.path.join(RUN_DIR, d) for d in os.listdir(RUN_DIR) if os.path.isdir(os.path.join(RUN_DIR, d))][0] # only one directory in the RUN_DIR
NEWS_REPORT = os.path.join(MAIN, 'news_report.txt')
SORAPROMPTS = os.path.join(MAIN, 'soraprompts.txt')
SCRIPT = os.path.join(MAIN, 'script.txt')
keyword = MAIN.split('\\')[-1].replace('_', ' ')

In [238]:
finalTrendList = pickle.load(open('./tmp/finalTrendList.pkl', 'rb'))

In [239]:
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]

In [240]:
with open(NEWS_REPORT) as f:
    newsReport = f.read()
print(newsReport)

**Title: Autopen Controversy Heats Up as Trump Challenges Biden's Pardons**

**Date: March 18, 2025**

In an escalating feud over the legitimacy of Joe Biden’s final pardons, former President Donald Trump accused his predecessor of signing these legal documents with an “autopen,” claiming that such actions render the pardons “void, vacant, and of no further force or effect.” 

Trump's assertions, posted on his Truth Social account, particularly target preemptive pardons granted to members of the House Select Committee investigating the January 6 Capitol insurrection. He stated, “In other words, Joe Biden did not sign them but, more importantly, he did not know anything about them!" This comment fueled speculation about Biden’s cognitive abilities and his control over the executive powers of the presidency.

Using the autopen, a mechanical device designed to replicate signatures, is a practice that has been common among U.S. presidents for decades. Legal experts and former governmental 

In [242]:
GROKPATH = MAIN + '/grok'
if not os.path.exists(GROKPATH):
    os.mkdir(GROKPATH)

In [243]:
soraPromptPath = GROKPATH + '/grok-soraprompts.txt'
soraPromptsv2Path = GROKPATH + '/grok-soraprompts-v2.txt'
scriptPath = GROKPATH + '/grok-script.txt'
grokImagePromptsPath = GROKPATH + '/grok-imageprompts.txt'
grokNarrationPath = GROKPATH + '/grok-narration.txt'
grokHashtagsPath = GROKPATH + '/grok-hashtags.txt'
opinionAnalysisPath = GROKPATH + '/grok-opinionanalysis.txt'

In [218]:
narrationWC = 0
while narrationWC < 60 or narrationWC > 80:
    script = generate_script_grok(grok, trend, newsReport)
    narration = generate_grok_narration(grok, script)
    narrationWC = len(narration.split(' '))
    print('narrationWC: ', narrationWC)
    print(narration)
    print("=========")
with open(scriptPath, 'w') as f:
    f.write(script)
with open(grokNarrationPath, 'w') as f:
    f.write(narration)

narrationWC:  63
Senate Democrats are in turmoil after ten of them, led by Chuck Schumer, voted with Republicans to pass a $1.7 trillion government funding bill, narrowly averting a shutdown. The bill, slammed by progressives like AOC as a "huge slap in the face," cuts non-defense spending while boosting military funds. Can the Democratic Party survive this deep rift as they head into crucial elections?


In [244]:
images = generate_grok_image_prompts(grok, script)
with open(grokImagePromptsPath, 'w') as f:
    f.write(images)
print(images)

Generate an image of Chuck Schumer standing at a crossroads, one path leading to a towering Capitol Building draped in Republican red, the other to a crumbling Democratic blue structure, symbolizing the party's internal conflict.

Generate an image of AOC, with a fiery expression, holding a sign that reads "Huge Slap in the Face," standing in front of a giant, exaggerated $1.7 trillion bill that overshadows her, representing her critique of the government funding bill.

Generate an image of a dramatic tug-of-war scene on the Senate floor, with ten Democrats, led by Chuck Schumer, pulling a rope labeled "$1.7 Trillion" towards a group of cheering Republicans, while other Democrats, looking dismayed, struggle to pull it back, illustrating the division within the party.

Generate an image of a giant military tank labeled "Increased Military Funds" rolling over a smaller, crushed car labeled "Non-Defense Spending," symbolizing the bill's impact on budget allocations.

Generate an image of 

In [220]:
soraPrompts = generate_sora_prompts_grok(grok, script)
with open(soraPromptPath, 'w') as f:
    f.write(soraPrompts[0])
print(soraPrompts[0])

[Senate in Turmoil]
[Senate floor, Democrats arguing, Schumer leading group to vote with Republicans]
[Pan across arguing senators, zoom in on Schumer]
 

[Funding Bill Details]
[Text overlay: "$1.7 trillion bill cuts non-defense, boosts military funds"]
[Text fades in, camera slowly zooms out to show full bill document]
 

[Progressive Reaction]
[Alexandria Ocasio-Cortez at podium, visibly upset, calling bill a "huge slap in the face"]
[Close-up on AOC, cut to reaction shots of audience]
 

[Democratic Party's Future]
[Split screen: Democratic Party symbol vs. upcoming election graphics]
[Camera pans from symbol to election graphics, fade to black]


In [222]:
soraPrompts2 = generate_sora_prompts_grokV2(grok, script)
with open(soraPromptsv2Path, 'w') as f:
    f.write(soraPrompts2[0])
print(soraPrompts2[0])

[Senate in Turmoil]
Senate Democrats argue, Schumer leads, Republicans join, voting on funding bill.
Camera pans across arguing senators, zooms on Schumer, cuts to vote.

[Funding Bill Details]
Visuals of bill text, highlighting military spending increase, non-defense cuts.
Camera scrolls through bill, focuses on key sections, fades to next scene.

[Progressive Reaction]
Alexandria Ocasio-Cortez speaks out, calling the bill a huge slap in the face.
Camera close-up on AOC, cuts to crowd reaction, pans to exit.

[Democratic Party's Future]
Democratic leaders in discussion, concerned looks, election posters in background.
Camera circles group, zooms on posters, fades out.


In [223]:
hashtags = generate_hashtags(grok, script)
with open(grokHashtagsPath, 'w') as f:
    f.write(hashtags)
print(hashtags)


1. [#SenateDemocrats] [This hashtag directly references the main subject of the video, the actions of Senate Democrats, which is crucial for targeting the relevant audience interested in political news and events.]
2. [#ChuckSchumer] [As a key figure in the narrative, mentioning Chuck Schumer by name will attract viewers interested in his political decisions and leadership within the Democratic Party.]
3. [#GovernmentFundingBill] [This hashtag specifies the central issue of the video, helping to attract viewers interested in government spending and fiscal policy.]
4. [#1_7Trillion] [Including the specific amount of the bill in the hashtag can draw attention from those interested in the scale of government spending and its implications.]
5. [#AOC] [Mentioning AOC, a prominent progressive critic of the bill, will engage viewers who follow her political stance and are interested in progressive viewpoints.]
6. [#MilitarySpending] [This hashtag targets viewers concerned with or interested i

In [224]:
opinionAnalysis = generate_grok_opinion_analysis(grok, newsReport)

In [225]:
with open(opinionAnalysisPath, 'w') as f:
    f.write(opinionAnalysis)
print(opinionAnalysis)

**Schumer's Betrayal: Democrats Divided and Defeated**

In a stunning display of political cowardice, Senate Democrats have fractured over a controversial funding bill, exposing the party's weakness and inability to stand united against Republican aggression. The passage of this $1.7 trillion spending bill, primarily backed by the GOP, is a clear victory for President Trump and his allies, including the ominously named Department of Government Efficiency led by Elon Musk.

Chuck Schumer and his nine Democratic colleagues who voted in favor of this bill have betrayed their party and their constituents. By siding with Republicans to keep the government running, they have handed Trump and his cronies the power to dismantle crucial federal agencies without oversight. Schumer's claim that a government shutdown would be worse is a feeble excuse for his capitulation to Republican demands.

The bill's $13 billion cut to non-defense spending and $6 billion increase in military funding is a slap